https://github.com/google/google-api-python-client
https://code.earthengine.google.com/
https://earthengine.google.com/
https://developers.google.com/earth-engine/python_install_manual

https://nbviewer.jupyter.org/github/python-visualization/folium_contrib/blob/master/notebooks/GoogleEarthEngine_layer.ipynb

In [20]:
import ee
import folium

ee.Initialize()
print(folium.__version__)

EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, and then retry.